<a href="https://colab.research.google.com/github/msrepo/ml-mscise-2023/blob/master/Lecture11_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from/Orginal notebook by: **Volodymyr Kuleshov**

## Part 1: The Kernel Trick: Motivation

So far, the majority of the machine learning models we have seen have been *linear*.

In this lecture, we will see a general way to make many of these models *non-linear*. We will use a new idea called *kernels*.

**Review: Linear Regression**

Recall that a linear model has the form
$$
 f(x) = \sum_{j=0}^d\theta_j\cdot x_j =\theta^\top x. 
$$
where $x$ is a vector of features and we used the notation $x_0 = 1$.

We pick $\theta$ to minimize the (L2-regularized) mean squared error (MSE). $$L(\theta) = \frac{1}{2n}\sum_{i=1}^n(y^{(i)} - \theta^\top x^{(i)}) + \frac{\lambda}{2}\sum_{j=1}^d\theta_j^2$$

**Review: Polynomial Regression**

Specifically, given a 1D continuous variable $x$, we can define a feature function $\phi: \mathbb{R} \to \mathbb{R}^{p+1}$ as 

$$
\phi(x) = 
\begin{bmatrix} 1 \\ x \\ x^2  \\ \vdots \\x^p \end{bmatrix} 
$$

The class of models of the form 

$$
f_\theta(x) :=\sum_{j=0}^p \theta_p x^p = \theta^\top \phi(x)
$$

**Towards general Non-Linear Features**

Any non-linear feature map $\phi(x):\mathbb{R} \to \mathbb{R}^p$ can be used to obtain general models of the form $f_\theta(x) := \theta^\top \phi(x)$ that are highly non-linear in $x$ but linear in $\theta$.

**The Featurized Design Matrix**

It is useful to represent the featurized dataset as a matrix $\Phi \in \mathbb{R}^{n \times p}$

**Featurized Normal Equations**

The normal equations provide a closed-form solution for $\theta$.

$$
\theta = (X^\top X + \lambda I)^{-1}X^\top y
$$

When the vectors of attributes $x^{(i)}$ are featurized, we can write this as 

$$
\theta = (\Phi^\top \Phi + \lambda I)^{-1}\Phi^\top y
$$

**Push-Through Matrix Identity**

We can modify this expression by using a version of the push-through matrix identity:

$$
(\lambda I + UV)^{-1}U = U(\lambda I + VU)^{-1}
$$

where $U \in \mathbb{R}^{n \times m}$ and $V \in \mathbb{R}^{m \times n}$ and $\lambda \neq 0$

Proof sketch: Start with $U (\lambda I + VU) = (\lambda I + UV)U$ and multiply both sides by $(\lambda I + VU)^{-1}$ on the right and $(\lambda I + UV)^{-1}$ on the left.